In [10]:
from utils import generate_samples_linear_reg
import numpy as np
import statsmodels.api as sm
import pandas as pd


In [5]:
samples = 10
features = 2
lower_bound = 0
upper_bound = 10
intercept = 5
coeffecients = [1,3]

X, y = generate_samples_linear_reg(
    samples,
    features,
    lower_bound,
    upper_bound,
    intercept,
    coeffecients,
)

print("Shape of samples is: ",X.shape)
print("Samples are: ", X)

print("\n Shape of labels is: ",y.shape)
print("Labels are: ", y)



Shape of samples is:  (10, 2)
Samples are:  [[7.09450541 0.0885049 ]
 [2.4260621  5.13349214]
 [7.6530514  9.61398845]
 [5.11898774 8.70824329]
 [9.81298224 8.85746227]
 [8.28453505 7.64397124]
 [0.77509095 5.98724072]
 [3.72212579 4.33538306]
 [4.99308537 2.63981972]
 [3.26831771 3.20579222]]

Shape of labels is:  (10,)
Labels are:  [12.3600201  22.82653851 41.49501674 36.2437176  41.38536904 36.21644877
 23.73681311 21.72827496 17.91254453 17.88569437]


In [16]:
X_with_intercept = sm.add_constant(X)
model = sm.OLS(y,X_with_intercept)
model_results = model.fit()
model_results.summary()

/opt/anaconda3/envs/linearregression/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.732e+30
Date:                Mon, 15 Apr 2024   Prob (F-statistic):          7.99e-106
Time:                        20:52:04   Log-Likelihood:                 308.38
No. Observations:                  10   AIC:                            -610.8
Df Residuals:                       7   BIC:                            -609.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.0000   9.49e-15   5.27e+14      0.000       5.000       5.000
x1             1.0000   1.45e-15   6.89e+14      0.000       1.000       1.000
x2             3.0000   1.33e-15   2.25e+15      0.000       3.000       3.000
==============================================================================
Omnibus:                        1.141   Durbin-Watson:                   1.066
Prob(Omnibus):                  0.565   Jarque-Bera (JB):                0.874
Skew:                          -0.581   Prob(JB):                        0.646
Kurtosis:                       2.135   Cond. No.                         21.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
a=np.empty((2,3))
a
a[0][0]=8
a

array([[8.00000000e+000, 3.11108246e+231, 1.42763657e-071],
       [1.85455425e-051, 1.18914750e-075, 4.17203572e-309]])

In [36]:
# np.array(([1,2,3],[3,4,5]))*np.array(([3],[4],[5]))

np.array(([1,2,3],[3,4,5]))*(np.array((3,4,5)))

# np.multiply(np.array(([1,2,3],[3,4,5]))*np.array(([3],[4],[5])))

array([[ 3,  8, 15],
       [ 9, 16, 25]])

In [38]:
np.dot(np.array(([1,2,3],[3,4,5])),(np.array((3,4,5))))

array([26, 50])

In [32]:
np.array((3,4,5)).T

array([3, 4, 5])